In [ ]:
# 데이터 크기 확인
print("X shape:", X.shape)
print("Y shape:", Y.shape)

# 데이터를 함께 섞기 위해 인덱스를 생성
num_samples = X.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

# 섞인 인덱스를 사용하여 데이터를 섞음
X_shuffled = X[indices]
Y_shuffled = Y[indices]

# 데이터를 9:1 비율로 train과 test 세트로 나눔
train_size = int(0.9 * num_samples)
X_train, X_test = X_shuffled[:train_size], X_shuffled[train_size:]
Y_train, Y_test = Y_shuffled[:train_size], Y_shuffled[train_size:]

# 결과 확인
print("Train set:")
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("Test set:")
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import gpflow
from gpflow.ci_utils import reduce_in_tests
from gpflow.config import default_float

## sample data
X = np.array(
    [
        [0.865], [0.666], [0.804], [0.771], [0.147], [0.866], [0.007], [0.026],
        [0.171], [0.889], [0.243], [0.028],
    ]
)
Y = np.array(
    [
        [1.57], [3.48], [3.12], [3.91], [3.07], [1.35], [3.80], [3.82], [3.49],
        [1.30], [4.00], [3.82],
    ]
)

# ## sample data - nosiy sinx
# num_train_data = reduce_in_tests(100)
# num_test_data = reduce_in_tests(500)

# X = tf.random.uniform((num_train_data, 1), dtype=default_float()) * 10
# Xtest = tf.random.uniform((num_test_data, 1), dtype=default_float()) * 10

# def noisy_sin(x):
#     return tf.math.sin(x) + 0.1 * tf.random.normal(
#         x.shape, dtype=default_float()
#     )
# Y = noisy_sin(X)
# Ytest = noisy_sin(Xtest)


# plt.plot(X, Y, "kx", mew=2)
# plt.show()

## create model
model = gpflow.models.GPR(
    (X, Y),
    kernel=gpflow.kernels.SquaredExponential(),
)

## train the model
opt = gpflow.optimizers.Scipy()
opt.minimize(model.training_loss, model.trainable_variables)

## wanna know f and Y might be at 0.5
Xnew = np.array([[0.5]])
model.predict_f(Xnew)
model.predict_y(Xnew)

## generate test pts for prediction
Xplot = np.linspace(-0.1, 1.1, 100)[:, None]
## predict f and Y
f_mean, f_var = model.predict_f(Xplot, full_cov=False)
y_mean, y_var = model.predict_y(Xplot)
## compute 95% confidence interval of f and Y
f_lower = f_mean - 1.96 * np.sqrt(f_var)
f_upper = f_mean + 1.96 * np.sqrt(f_var)
y_lower = y_mean - 1.96 * np.sqrt(y_var)
y_upper = y_mean + 1.96 * np.sqrt(y_var)

In [5]:
f_mean.shape

TensorShape([100, 1])